In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import torch
import torch.nn as nn
import torch.nn.functional as F


In [3]:
df_weather = pd.read_csv('weather_pca_data.csv')

In [4]:
df_train = pd.read_csv('for_students/waiting_times_train.csv')
df_train

,DATETIME,ENTITY_DESCRIPTION_SHORT,ADJUST_CAPACITY,DOWNTIME,CURRENT_WAIT_TIME,TIME_TO_PARADE_1,TIME_TO_PARADE_2,TIME_TO_NIGHT_SHOW,WAIT_TIME_IN_2H
0,2022-02-05 11:45:00,Water Ride,247.00,0,20,NaN,NaN,NaN,30.0
1,2019-02-24 10:45:00,Water Ride,247.00,0,30,375.0,NaN,495.0,25.0
2,2021-07-17 15:45:00,Pirate Ship,280.50,0,35,NaN,NaN,NaN,35.0
3,2022-04-03 19:45:00,Pirate Ship,230.35,0,15,-135.0,NaN,195.0,10.0
4,2021-10-20 10:30:00,Pirate Ship,153.00,0,15,NaN,NaN,NaN,10.0
...,...,...,...,...,...,...,...,...,...
37013,2022-05-12 15:30:00,Flying Coaster,756.00,0,30,120.0,NaN,450.0,10.0
37014,2018-10-17 13:15:00,Pirate Ship,280.50,0,15,255.0,-65.0,405.0,20.0
37015,2018-12-27 19:45:00,Flying Coaster,756.00,0,15,-165.0,-475.0,135.0,10.0
37016,2020-02-16 13:00:00,Flying Coaster,756.00,0,30,240.0,NaN,420.0,45.0


In [5]:
dates = df_train["DATETIME"]
df_train.drop(["DATETIME", "ENTITY_DESCRIPTION_SHORT"], axis=1, inplace=True)
df_train=(df_train-df_train.mean())/df_train.std()
df_train["DATETIME"] = dates
df_train

,ADJUST_CAPACITY,DOWNTIME,CURRENT_WAIT_TIME,TIME_TO_PARADE_1,TIME_TO_PARADE_2,TIME_TO_NIGHT_SHOW,WAIT_TIME_IN_2H,DATETIME
0,-0.660639,-0.168584,-0.240147,NaN,NaN,NaN,0.447712,2022-02-05 11:45:00
1,-0.660639,-0.168584,0.468341,1.045710,NaN,0.341594,0.097626,2019-02-24 10:45:00
2,-0.527907,-0.168584,0.822585,NaN,NaN,NaN,0.797797,2021-07-17 15:45:00
3,-0.726608,-0.168584,-0.594390,-1.580494,NaN,-1.133824,-0.952630,2022-04-03 19:45:00
4,-1.033079,-0.168584,-0.594390,NaN,NaN,NaN,-0.952630,2021-10-20 10:30:00
...,...,...,...,...,...,...,...,...
37013,1.356085,-0.168584,0.468341,-0.267392,NaN,0.120281,-0.952630,2022-05-12 15:30:00
37014,-0.527907,-0.168584,-0.594390,0.427780,0.388351,-0.101031,-0.252459,2018-10-17 13:15:00
37015,1.356085,-0.168584,-0.594390,-1.734976,-1.838470,-1.428907,-0.952630,2018-12-27 19:45:00
37016,1.356085,-0.168584,0.468341,0.350538,NaN,-0.027261,1.497968,2020-02-16 13:00:00


In [7]:
date = df_train["DATETIME"][0]
df_weather.loc[df_weather["DATETIME"] == date]

,PC1,PC2,PC3,PC4,PC5,PC6,DATETIME
117455,-1.703404,-0.561189,-0.521895,0.099553,0.564666,0.303006,2022-02-05 11:45:00


In [8]:
df_combined = pd.merge(df_train, df_weather, on='DATETIME', how='left')

In [9]:
df_combined

,ADJUST_CAPACITY,DOWNTIME,CURRENT_WAIT_TIME,TIME_TO_PARADE_1,TIME_TO_PARADE_2,TIME_TO_NIGHT_SHOW,WAIT_TIME_IN_2H,DATETIME,PC1,PC2,PC3,PC4,PC5,PC6
0,-0.660639,-0.168584,-0.240147,NaN,NaN,NaN,0.447712,2022-02-05 11:45:00,-1.703404,-0.561189,-0.521895,0.099553,0.564666,0.303006
1,-0.660639,-0.168584,0.468341,1.045710,NaN,0.341594,0.097626,2019-02-24 10:45:00,-0.983224,-3.370968,1.688497,0.822912,-0.257597,1.030229
2,-0.527907,-0.168584,0.822585,NaN,NaN,NaN,0.797797,2021-07-17 15:45:00,1.971007,0.491093,-0.446060,-1.375623,0.711029,0.848786
3,-0.726608,-0.168584,-0.594390,-1.580494,NaN,-1.133824,-0.952630,2022-04-03 19:45:00,-1.153047,-1.460163,0.761943,0.267117,-0.799023,1.794617
4,-1.033079,-0.168584,-0.594390,NaN,NaN,NaN,-0.952630,2021-10-20 10:30:00,0.644351,2.261365,0.670276,0.539488,0.961583,0.083199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37013,1.356085,-0.168584,0.468341,-0.267392,NaN,0.120281,-0.952630,2022-05-12 15:30:00,2.474459,-0.242590,-1.250092,0.433454,-2.257249,1.038954
37014,-0.527907,-0.168584,-0.594390,0.427780,0.388351,-0.101031,-0.252459,2018-10-17 13:15:00,1.451294,-0.740837,-0.309993,-0.850829,1.254196,-0.054063
37015,1.356085,-0.168584,-0.594390,-1.734976,-1.838470,-1.428907,-0.952630,2018-12-27 19:45:00,-3.211167,-0.929818,-1.205296,-0.270431,0.000201,0.500882
37016,1.356085,-0.168584,0.468341,0.350538,NaN,-0.027261,1.497968,2020-02-16 13:00:00,0.588498,2.259265,2.034889,-0.714293,-0.254688,0.621635
